In [71]:
from skimage.metrics import structural_similarity as ssim
import cv2
#Structural Similarity Index
def SSIM_two_images(img1, img2): 
    gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    ssim_value = ssim(gray_img1, gray_img2)

    # if ssim_value == 1:
    #     print("SSIM: The images are identical")
    # else:
    #     print("SSIM: The images are not identical")
    return ssim_value

In [72]:
# import cv2
def  MSE_two_images(img1, img2):
    # gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    # gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    mse = ((img1 - img2) ** 2).mean(axis=None)

    # if mse == 0:
    #     print("MSE: The images are identical")
    # else:
    #     print("MSE: The images are not identical")

    return mse

In [115]:
def To_got_goodFeaturesToTrack(image):

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Applying the function
    corners = cv2.goodFeaturesToTrack(
        gray_image, maxCorners=8192, qualityLevel=0.04, minDistance=9, blockSize=9)
    corners = np.float32(corners)
    # print(len(corners))
    for item in corners:
        x, y = item[0]
        x = int(x)
        y = int(y)
        cv2.circle(image, (x, y), 3, (0, 0, 255), -1)

    return image
    # RGB_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
#   Pyramid Lucas-Kanade optical flow
def To_got_optical_flow(image1, image2):
    

In [118]:
import cv2
import numpy as np

# vidcap = cv2.VideoCapture('Data\Videos\Aerial View of the Great Pyramid of Giza.mp4')
vidcap = cv2.VideoCapture('Data\Videos\wymont.MOV')
success,image = vidcap.read()
print(image.shape) 
print('Height of Image:', int(image.shape[0]), 'pixels')
print('Width of Image: ', int(image.shape[1]), 'pixels')
count = 0
old_image = image
while success:

    #cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file
    cv2.imshow('image', To_got_goodFeaturesToTrack(image))      # show frame
    # print('MSE: ', MSE_two_images(image, old_image)) # compare two images
    # print('SSIM: ', SSIM_two_images(image, old_image)) # compare two images
    # gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    old_image = image
    k = cv2.waitKey(1)
    if k == 27:         # wait for ESC key to exit
        cv2.destroyAllWindows()
        break
    success,image = vidcap.read()   # read next frame
    # print('Read a new frame: ', success, count)
    count += 1

cv2.destroyAllWindows()
vidcap.release()

(1080, 1920, 3)
Height of Image: 1080 pixels
Width of Image:  1920 pixels


https://sandipanweb.wordpress.com/2018/02/25/implementing-lucas-kanade-optical-flow-algorithm-in-python/

In [ ]:
# how to calculate the frame rate with python opencv
# https://www.geeksforgeeks.org/how-to-calculate-the-frame-rate-with-python-opencv/

In [ ]:
import numpy as np
import cv2 as cv
import math
cap = cv.VideoCapture('Data\Videos\wymont.MOV')
# cap = cv.VideoCapture('Data\Videos\Aerial View of the Great Pyramid of Giza.mp4')
#角点检测参数
# feature_params = dict(maxCorners=128, qualityLevel=0.1, minDistance=7, blockSize=7)
feature_params = dict(maxCorners=512, qualityLevel=0.04, minDistance=9, blockSize=9)


#KLT光流参数
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.02))

height = cap.get(cv.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv.CAP_PROP_FPS)
#out = cv.VideoWriter("reslut.avi", cv.VideoWriter_fourcc('D', 'I', 'V', 'X'), fps,
                     #(np.int(width), np.int(height)), True)

tracks = []
track_len = 10
frame_idx = 0
detect_interval = 10
while True:

    ret, frame = cap.read()
    if ret:
        frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        vis = frame.copy()

        if len(tracks)>0:
            img0 ,img1 = prev_gray, frame_gray
            p0 = np.float32([tr[-1] for tr in tracks]).reshape(-1,1,2)
            # 上一帧的角点和当前帧的图像作为输入来得到角点在当前帧的位置  
            p1, st, err = cv.calcOpticalFlowPyrLK(img0, img1, p0, None, **lk_params)

            # 反向检查,当前帧跟踪到的角点及图像和前一帧的图像作为输入来找到前一帧的角点位置  
            p0r, _, _ = cv.calcOpticalFlowPyrLK(img1, img0, p1, None, **lk_params)

            # 得到角点回溯与前一帧实际角点的位置变化关系 
            d = abs(p0-p0r).reshape(-1,2).max(-1)

            #判断d内的值是否小于1，大于1跟踪被认为是错误的跟踪点
            good = d < 1

            new_tracks = []

            for i, (tr, (x, y), flag) in enumerate(zip(tracks, p1.reshape(-1, 2), good)):

                # 判断是否为正确的跟踪点
                if not flag:
                    continue

                # 存储动态的角点
                tr.append((x, y))

                # 只保留track_len长度的数据，消除掉前面的超出的轨迹
                if len(tr) > track_len:
                    del tr[0]
                # 保存在新的list中
                new_tracks.append(tr)

                # cv.circle(vis, (x, y), 2, (0, 255, 0), -1)
                cv.circle(vis, (int(x), int(y)), 2, (0, 255, 0), -1)

            # 更新特征点    
            tracks = new_tracks

            # #以上一振角点为初始点，当前帧跟踪到的点为终点,画出运动轨迹
            cv.polylines(vis, [np.int32(tr) for tr in tracks], False, (0, 0, 255), 1)
            # cv.circle(vis, [np.int32(tr) for tr in tracks], False, (255, 0, 0), 1)


        # 每隔 detect_interval 时间检测一次特征点
        if frame_idx % detect_interval==0:
            # mask = np.zeros_like(frame_gray)
            # mask[:] = 255

            # if frame_idx !=0:
            #     for x,y in [np.int32(tr[-1]) for tr in tracks]:
            #         cv.circle(mask, (x, y), 5, 0, -1)

            # p = cv.goodFeaturesToTrack(frame_gray, mask=mask, **feature_params)
            p = cv.goodFeaturesToTrack(frame_gray, **feature_params)
            if p is not None:
                for x, y in np.float32(p).reshape(-1,2):
                    tracks.append([(x, y)])

        frame_idx += 1
        prev_gray = frame_gray

        cv.imshow('track', vis)
        #out.write(vis)
        ch = cv.waitKey(1)
        if ch ==27:
            # cv.imwrite('track.jpg', vis)
            break
    else:
        break

cv.destroyAllWindows()
cap.release()

https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html

In [13]:
import numpy as np
import cv2 as cv
import argparse

# parser = argparse.ArgumentParser(description='This sample demonstrates Lucas-Kanade Optical Flow calculation. \
#                                               The example file can be downloaded from: \
#                                               https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4')
# parser.add_argument('image', type=str, help='path to image file')
# args = parser.parse_args()
# cap = cv.VideoCapture(args.image)
# cap = cv.VideoCapture('Data\Videos\wymont.MOV')
cap = cv.VideoCapture('Data\Videos\Aerial View of the Great Pyramid of Giza.mp4')
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None:
        good_new = p1[st==1]
        good_old = p0[st==1]

    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv.add(frame, mask)

    cv.imshow('frame', img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv.destroyAllWindows()

In [4]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture("Data\Videos\wymont.MOV")
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while(1):
    ret, frame2 = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png', frame2)
        cv.imwrite('opticalhsv.png', bgr)
    prvs = next

cv.destroyAllWindows()